In [2]:
# Import modules
import datetime
import spiceypy
import numpy as np
import pandas as pd

# Clear any previously loaded kernels
spiceypy.kclear()

# Load kernels via meta file
spiceypy.furnsh('kernel_meta.txt')
print("Meta Kernel Loaded:", spiceypy.ktotal("ALL"))

# Manually load kernels (in correct order)
spiceypy.furnsh('../kernels/lsk/naif0012.tls')  # Leap Seconds
spiceypy.furnsh('../kernels/spk/de432s.bsp')    # Ephemeris Data
spiceypy.furnsh('../kernels/pck/pck00010.tpc')  # Planet Constants

# Verify total loaded kernels
print("Total Kernels Loaded:", spiceypy.ktotal("ALL"))

# Create an initial and ending time date-time object that is converted to a
# string
init_time_utc_str = datetime.datetime(year=2021, month=10, day=1).strftime('%Y-%m-%dT%H:%M:%S')
end_time_utc_str = datetime.datetime(year=2022, month=6, day=1).strftime('%Y-%m-%dT%H:%M:%S')

# Convert to Ephemeris Time (ET) using the SPICE function utc2et
init_time_et = spiceypy.utc2et(init_time_utc_str)
end_time_et = spiceypy.utc2et(end_time_utc_str)

# Set the number of seconds per hours. This value is used to compute the phase
# angles in 1 hour steps (the ET is given in seconds)
delta_hour_in_seconds = 3600.0
time_interval_et = np.arange(init_time_et, end_time_et, delta_hour_in_seconds)

Meta Kernel Loaded: 1
Total Kernels Loaded: 4


In [3]:
# All our computed parameters, positions etc. shall be stored in a pandas
# dataframe. First, we create an empty one
inner_solsys_df = pd.DataFrame()

# Set the column ET that stores all ETs
inner_solsys_df.loc[:, 'ET'] = time_interval_et

# The column UTC transforms all ETs back to a UTC format. The function
# spicepy.et2datetime is NOT an official part of SPICE (there you can find
# et2utc).
# However this function returns immediately a date-time object
inner_solsys_df.loc[:, 'UTC'] = \
    inner_solsys_df['ET'].apply(lambda x: spiceypy.et2datetime(et=x))